In [80]:
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
import os
import json
import requests
from typing import Any, List, Mapping, Optional

from langchain_core.documents import Document
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [81]:
class FreeLLM(LLM):
    n: int
    model_to_use: str = "deepseek/deepseek-r1-0528:free" # Default model

    @property
    def _llm_type(self) -> str:
        return "deepseek/deepseek-r1-0528:free"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')
        if not OPENROUTER_API_KEY:
            raise ValueError("OPENROUTER_API_KEY not found in environment variables.")
        YOUR_SITE_URL = 'https://github.com/billabavandar/testBot'
        headers = {
            'Authorization': f'Bearer {OPENROUTER_API_KEY}',
            'HTTP-Referer': YOUR_SITE_URL,
            'X-Title': 'LangChain Learning Project',
            'Content-Type': 'application/json'
        }
        data = {
            'model': self.model_to_use,
            'messages': [{'role': 'user', 'content': prompt}]
        }

        response = requests.post('https://openrouter.ai/api/v1/chat/completions', headers=headers, data=json.dumps(data))

        if response.status_code != 200:
            raise ValueError(f"OpenRouter API request failed: {response.status_code} - {response.text}")
        try:
            response_json = response.json()
        except requests.exceptions.JSONDecodeError:
            raise ValueError(f"Failed to decode JSON from OpenRouter response: {response.text}")

        try:
            output = response_json['choices'][0]['message']['content']
        except (KeyError, IndexError, TypeError) as e:
            print(f"Error parsing OpenRouter response structure: {e}")
            print(f"Full JSON response was: {response_json}")
            raise

        if stop is not None:
            # Currently not passing stop to API, this is a placeholder
            print("Warning: 'stop' arguments were passed but are not implemented in this custom LLM.")
        return output

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"n": self.n, "model_name": self.model_to_use}

In [82]:
documents_folder_path = "./Advisories"

loader_pdf = DirectoryLoader(
    path = documents_folder_path,
    glob = "*.pdf",
    loader_cls = PyPDFLoader,
    show_progress = True,
    use_multithreading = False
)

In [83]:
all_loaded_docs = []

pdf_docs = loader_pdf.load()
all_loaded_docs.extend(pdf_docs)

 16%|█▌        | 6/38 [00:00<00:00, 49.90it/s]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 69 0 (offset 0)
Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong pointing object 83 0 (offset 0)
Ignoring wrong pointing object 85 0 (offset 0)
Ignoring wrong pointing object 112 0 (offset 0)
Ignoring wrong pointing object 114 0 (offset 0)
Ignoring wrong pointing object 116 0 (offset 0)
Ignoring wrong pointing object 119 0 (offset 0)
Ignoring wrong pointing object 121 0 (offset 0)
Ignoring wron

In [84]:
lmworks = 0
try:
    llm_for_all = FreeLLM(n=1, model_to_use = "deepseek/deepseek-r1-0528:free")
except:
    lmworks = 1
    print("llm dont work")

In [110]:
if not lmworks:
    token_memory = ConversationTokenBufferMemory(
        llm = llm_for_all,
        max_token_limit = 300,
        memory_key = 'hist',
        return_messages = True,
        input_key = "input",
        output_key = "output",
    )

In [111]:
#_template = "This is a conversation between an AI and a student trying to know more about the college IIT Gandhinagar, the AI is mildly talkative and provides lots of specific details from it's contents, if the AI does not know the answer to a question, it truthfully says it does not know. Current conversation: {hist}. Human: {input}. AI:"

prompt = ChatPromptTemplate.from_messages([("system", "This is a conversation between an AI and a student trying to know more about the college IIT Gandhinagar, the AI is mildly talkative and provides lots of specific details from it's contents, if the AI does not know the answer to a question, it truthfully says it does not know."),
    MessagesPlaceholder(variable_name = "hist"),
                                           ("human", "{input}")])

In [112]:
core_runnable = prompt | llm_for_all

In [113]:
runnable = RunnableWithMessageHistory(
    prompt | llm_for_all,
    lambda session_id: token_memory,
    input_messages_key = "input",
    output_messages_key = "output",
    history_messages_key = "hist",
)

In [114]:
session_id = "my_iit_gn_chat_session"
interactions = [
        "Hello! I'm interested in IIT Gandhinagar. Can you tell me about its campus life?",
        "That sounds interesting. What about research opportunities for undergraduate students?",
        "Thanks! Do you remember what I first asked you about?", # Test memory
        "What's the weather like there usually?" # A general knowledge question
    ]


In [115]:
for i, user_input_text in enumerate(interactions):
        print(f"\n--- Interaction {i+1} ---")
        print(f"Student: {user_input_text}")

        response_content = runnable.invoke(
            {"input": input},
            config={"configurable": {"session_id": session_id}}
        )
        print(f"AI: {response_content}")


--- Interaction 1 ---
Student: Hello! I'm interested in IIT Gandhinagar. Can you tell me about its campus life?


AttributeError: 'ConversationTokenBufferMemory' object has no attribute 'messages'